In [1]:
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import stopwords
import nltk
import numpy as np
import pandas as pd
import preprocessor as tpre
import os
import glob

In [2]:
# preperation
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('sentiwordnet')
nltk.download('stopwords')
stopset = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /Users/cary/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/cary/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /Users/cary/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/cary/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Where stores the individual data
soldierCalDict = {}

In [4]:
# file list
soldierCSVs = []
files = os.listdir(os.getcwd() + "/data/soldiers")
for file in files:
     if not os.path.isdir(file) and file.endswith(".csv"):
            soldierCSVs.append(os.getcwd() + "/data/soldiers/" + file)

In [5]:
tempSoldier = soldierCSVs[0]

In [6]:
tempSoldier

'/Users/cary/Work/TextAnalytics/supporting_scripts/data/soldiers/fcharles81.csv'

In [7]:
tempDf = pd.read_csv(tempSoldier)

In [8]:
tempDf.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1242638918434111494,1242636719276593153,1585103179000,2020-03-25,02:26:19,IST,37905210,fcharles81,Cisco,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '37905210', 'username': 'fcharles...",NaN,NaN,NaN,NaN
1,1241569382347636736,1241457607811964935,1584848182000,2020-03-22,03:36:22,IST,37905210,fcharles81,Cisco,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '37905210', 'username': 'fcharles...",NaN,NaN,NaN,NaN
2,1241557835407921152,1241457607811964935,1584845429000,2020-03-22,02:50:29,IST,37905210,fcharles81,Cisco,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '37905210', 'username': 'fcharles...",NaN,NaN,NaN,NaN
3,1241419057674498048,1241419057674498048,1584812342000,2020-03-21,17:39:02,IST,37905210,fcharles81,Cisco,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '37905210', 'username': 'fcharles...",NaN,NaN,NaN,NaN
4,1241206985833492483,1056886250387464192,1584761780000,2020-03-21,03:36:20,IST,37905210,fcharles81,Cisco,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '37905210', 'username': 'fcharles...",NaN,NaN,NaN,NaN


In [9]:
tempDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2483 entries, 0 to 2482
Data columns (total 34 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               2483 non-null   int64  
 1   conversation_id  2483 non-null   int64  
 2   created_at       2483 non-null   int64  
 3   date             2483 non-null   object 
 4   time             2483 non-null   object 
 5   timezone         2483 non-null   object 
 6   user_id          2483 non-null   int64  
 7   username         2483 non-null   object 
 8   name             2483 non-null   object 
 9   place            0 non-null      float64
 10  tweet            2483 non-null   object 
 11  mentions         2483 non-null   object 
 12  urls             2483 non-null   object 
 13  photos           2483 non-null   object 
 14  replies_count    2483 non-null   int64  
 15  retweets_count   2483 non-null   int64  
 16  likes_count      2483 non-null   int64  
 17  hashtags      

In [10]:
# useless columns for analyzing
uselessColumns = [
    "id",
    "conversation_id",
    "place",
    "photos",
    "video",
    "near",
    "geo",
    "source",
    "user_rt_id",
    "user_rt",
    "retweet_id",
    "reply_to",
    "retweet_date",
    "translate",
    "trans_src",
    "trans_dest",
    "link"
]


In [11]:
tempDf.dropna(subset=['tweet'], inplace=True)
tempDf.drop(uselessColumns, axis=1, inplace=True)

In [12]:
tempDf.head()

,created_at,date,time,timezone,user_id,username,name,tweet,mentions,urls,replies_count,retweets_count,likes_count,hashtags,cashtags,retweet,quote_url
0,1585103179000,2020-03-25,02:26:19,IST,37905210,fcharles81,Cisco,Same,['callinorton'],[],0,0,0,[],[],False,NaN
1,1584848182000,2020-03-22,03:36:22,IST,37905210,fcharles81,Cisco,Thanks!!,['overclockthis'],[],0,0,1,[],[],False,NaN
2,1584845429000,2020-03-22,02:50:29,IST,37905210,fcharles81,Cisco,What website is this?,['overclockthis'],[],1,0,1,[],[],False,NaN
3,1584812342000,2020-03-21,17:39:02,IST,37905210,fcharles81,Cisco,Gettin' crazy out there https://twitter.com/Re...,[],['https://twitter.com/RealMattCouch/status/124...,1,0,1,[],[],False,https://twitter.com/RealMattCouch/status/12411...
4,1584761780000,2020-03-21,03:36:20,IST,37905210,fcharles81,Cisco,pic.twitter.com/xC4SQSf8gc,['acosta'],[],0,0,0,[],[],False,NaN


In [13]:
# convert pos from nltk.pos_tag to SentiWordNet
# n - NOUN 
# v - VERB 
# a - ADJECTIVE 
# s - ADJECTIVE SATELLITE 
# r - ADVERB
# ' ' - others
# a list of nltk tags is here:
# https://www.techrepublic.com/article/the-6-laws-every-cloud-architect-should-know-according-to-werner-vogels/
def sentiWordNetPOSconvetor(pos):
    newtag = ''
    if pos.startswith('NN'):
        newtag='n'
    elif pos.startswith('JJ'):
        newtag='a'
    elif pos.startswith('V'):
        newtag='v'
    elif pos.startswith('R'):
        newtag='r'
    return newtag

In [21]:
# word: single word
# pos: part of speech from nltk.pos_tag
def calSentiForWord(word, pos):
    
    sentis = list(swn.senti_synsets(word, sentiWordNetPOSconvetor(pos)))
    
    if len(sentis) <= 0:
        return 0, 0, 0
    
    # Getting average of all possible sentiments
    positive = 0.0
    negative = 0.0
    objective = 0.0
    count = 0.0
    
    for senti in sentis :
        positive += senti.pos_score()
        negative += senti.neg_score()
        objective += senti.obj_score()
        count += 1
    
    positive /= count
    negative /= count
    objective /= count
    
    return positive, negative, objective

In [22]:
def calSentiForTweet(tweet):
    tempTweet = tweet
    tpre.set_options(tpre.OPT.URL, tpre.OPT.EMOJI)
    tempTweetCleaned = tpre.clean(tempTweet)
    
    if len(tempTweetCleaned) <= 0:
        return 0, 0, 0
    
    tempTokens = nltk.word_tokenize(tempTweetCleaned)
    tempTokens = [tempToken.lower() for tempToken in tempTokens if len(tempToken)>2]
#     tempTokens = [tempToken.lower() for tempToken in tempTokens if tempToken not in stopset and len(tempToken)>2]
    tempTagged = nltk.pos_tag(tempTokens)
    
    positive = 0.0
    negative = 0.0
    objective = 0.0
    count = 0.0
    
#     print(tempTagged)
    
    for w, p in tempTagged:
        po, ne, ob = calSentiForWord(w, p)
        positive += po
        negative += ne
        objective += ob
        count += 1
        
    positive /= count
    negative /= count
    objective /= count
    
    return positive, negative, objective

In [23]:
print(calSentiForTweet(tempDf.loc[0, "tweet"]))

[('same', 'JJ')]
(0.125, 0.15625, 0.71875)
